In [6]:
pip install statsmodels

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, r2_score

In [8]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = 'Fact_Customer_Orders'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [9]:
df.columns

Index(['ORDER_ID', 'PRODUCT_ID', 'ORDER_QTY', 'ORDER_DATE'], dtype='object')

In [ ]:
data = df[[PRODUCT_ID','ORDER_QTY', 'ORDER_DATE']]

In [11]:
data['ORDER_DATE'] = pd.to_datetime(data['ORDER_DATE'])

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225563 entries, 0 to 225562
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   ORDER_ID    225563 non-null  object        
 1   PRODUCT_ID  225563 non-null  object        
 2   ORDER_QTY   225563 non-null  int16         
 3   ORDER_DATE  225563 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int16(1), object(2)
memory usage: 5.6+ MB


In [13]:
data

,ORDER_ID,PRODUCT_ID,ORDER_QTY,ORDER_DATE
0,Order_000006,Product_001,850,2019-01-02
1,Order_000001,Product_004,700,2019-01-02
2,Order_000003,Product_004,900,2019-01-02
3,Order_000008,Product_004,550,2019-01-02
4,Order_000012,Product_004,300,2019-01-02
...,...,...,...,...
225558,Order_161185,Product_098,200,2024-08-31
225559,Order_161269,Product_100,800,2024-08-31
225560,Order_161275,Product_100,550,2024-08-31
225561,Order_161283,Product_100,850,2024-08-31


In [14]:
data = data.set_index('ORDER_DATE')

In [15]:
data

,ORDER_ID,PRODUCT_ID,ORDER_QTY
ORDER_DATE,,,
2019-01-02,Order_000006,Product_001,850
2019-01-02,Order_000001,Product_004,700
2019-01-02,Order_000003,Product_004,900
2019-01-02,Order_000008,Product_004,550
2019-01-02,Order_000012,Product_004,300
...,...,...,...
2024-08-31,Order_161185,Product_098,200
2024-08-31,Order_161269,Product_100,800
2024-08-31,Order_161275,Product_100,550


In [16]:
data.sort_index(inplace=True)

In [17]:
data

,ORDER_ID,PRODUCT_ID,ORDER_QTY
ORDER_DATE,,,
2019-01-02,Order_000006,Product_001,850
2019-01-02,Order_000001,Product_004,700
2019-01-02,Order_000003,Product_004,900
2019-01-02,Order_000008,Product_004,550
2019-01-02,Order_000012,Product_004,300
...,...,...,...
2024-08-31,Order_161185,Product_098,200
2024-08-31,Order_161269,Product_100,800
2024-08-31,Order_161275,Product_100,550


In [18]:
# Group data by 'product_id'
grouped_data = data.groupby('PRODUCT_ID')

In [19]:
# Initialize an empty DataFrame for forecast results
test_results = pd.DataFrame(columns=['PRODUCT_ID','ORDER_DATE', 'test_forecast_order_qty','rse', 'rmse','r2_score'])

In [20]:
# Initialize an empty DataFrame for forecast results
forecast_results = pd.DataFrame(columns=['PRODUCT_ID','ORDER_DATE', 'forecast_order_qty'])